In [11]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
data = pd.read_csv("C:\\Users\\krist\\Downloads\\NUS Fintech Society\\Project\\NUS-FinTech-Society-Project\\project.ipynb\\dogecoin.csv")
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1095 entries, 0 to 1094
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1095 non-null   object 
 1   Close   1094 non-null   float64
dtypes: float64(1), object(1)
memory usage: 17.2+ KB


In [4]:
# Data processing 
# convert date column into data time type 
data['Date'] = pd.to_datetime(data["Date"])
# Sort the DataFrame by the 'Date' column in ascending order
data = data.sort_values(by='Date', ascending=True)



In [5]:
# Create Target column ( Only start to have values from Day 8 onwards )
data['Target'] = np.nan
data.loc[7:, 'Target'] = (data['Close'].diff() > 0).astype(int)
data["Target"] = data["Target"].astype('category')


In [6]:
def create_features(data, target):
    """
    Parameters:
    - data: A pandas Series containing the feature data (e.g.,
    Closing prices).
    - target: A pandas Series containing the target variable
    (e.g., the column indicating up or down).
    Returns:
    - A tuple containing two numpy arrays: features with
    matching shapes for LSTM (X): (None, 7, 1) and target (y).
    """
    X, y = [], []
    for i in range(len(data) - 7):
        X.append(data[i:i + 7].values)
        y.append(target[i + 7])
    X = np.array(X)
    return X.reshape((X.shape[0], X.shape[1], 1)), np.array(y)

In [7]:
# Create x and y ( to prepare for model training )
data_x = data["Close"]
target = data["Target"]
X,y = create_features(data_x,target)
# In y: 0 represents that the price on a given day is lower than the previous day 
# 1 represents that the price on a given day is higher than the previous day 



In [ ]:
# Model training 

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False)

# Standardise the X_train and X_val separately
scaler = StandardScaler()
X_train_reshaped = X_train.reshape(-1, 1)
X_train_scaled = scaler.fit_transform(X_train_reshaped).reshape(X_train.shape)
X_val_reshaped = X_val.reshape(-1, 1)
X_val_scaled = scaler.transform(X_val_reshaped).reshape(X_val.shape)



In [ ]:
# Building neural network 
model = Sequential()
model.add(LSTM(50, input_shape=(7, 1), return_sequences=False))
model.add(Dropout(0.2))  # Dropout for regularization
model.add(Dense(1, activation='sigmoid'))  # Sigmoid for binary classification

